In [ ]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="BQvvxKrJPoAaNkv4ZXlr")
project = rf.workspace("devflo").project("rdd2022-ook7x")
version = project.version(3)
dataset = version.download("yolov12")


from roboflow import Roboflow
rf = Roboflow(api_key="BQvvxKrJPoAaNkv4ZXlr")
project = rf.workspace("laura-hjwf7").project("pothole-at-night")
version = project.version(1)
dataset = version.download("yolov12")

import os
import shutil
import glob
import random
import math
import yaml

# ------------------ הגדרות ראשוניות ------------------
# נתיבים לדאטה:
new_dataset_dir = "/kaggle/working/Pothole-At-Night-1"  # הדאטה החדש
old_dataset_dir = "/kaggle/working/RDD2022-3"          # הדאטה הישן (אליו נעביר את הקבצים)

splits = ["train", "valid", "test"]

# ------------------ שלב 1: עדכון תיוגים בדאטה החדש ------------------
print("Step 1: Updating labels in new dataset...")

for split in splits:
    labels_dir = os.path.join(new_dataset_dir, split, "labels")
    if not os.path.isdir(labels_dir):
        print(f"Labels directory not found for split '{split}'. Skipping...")
        continue
    for filename in os.listdir(labels_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(labels_dir, filename)
            with open(file_path, "r") as f:
                lines = f.readlines()
            updated_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:  # פורמט YOLO: <class> <x_center> <y_center> <width> <height>
                    parts[0] = "8"  # שינוי ה-class_id ל-8
                    updated_lines.append(" ".join(parts) + "\n")
                else:
                    updated_lines.append(line)
            with open(file_path, "w") as f:
                f.writelines(updated_lines)
            print(f"Updated labels in {file_path}")
print("All labels in new dataset updated to class_id=8.\n")

# ------------------ שלב 2: מיזוג הדאטה ------------------
print("Step 2: Merging new dataset into old dataset...")

for split in splits:
    new_images_dir = os.path.join(new_dataset_dir, split, "images")
    new_labels_dir = os.path.join(new_dataset_dir, split, "labels")

    old_images_dir = os.path.join(old_dataset_dir, split, "images")
    old_labels_dir = os.path.join(old_dataset_dir, split, "labels")

    # ודא שהתיקיות קיימות ביעד
    os.makedirs(old_images_dir, exist_ok=True)
    os.makedirs(old_labels_dir, exist_ok=True)

    # העתקת תמונות מהדאטה החדש
    if os.path.isdir(new_images_dir):
        for file in os.listdir(new_images_dir):
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                src_path = os.path.join(new_images_dir, file)
                dst_path = os.path.join(old_images_dir, file)
                shutil.copy(src_path, dst_path)
                print(f"Copied image {src_path} -> {dst_path}")
    else:
        print(f"Images directory not found for split '{split}' in new dataset.")

    # העתקת תיוגים מהדאטה החדש
    if os.path.isdir(new_labels_dir):
        for file in os.listdir(new_labels_dir):
            if file.endswith(".txt"):
                src_path = os.path.join(new_labels_dir, file)
                dst_path = os.path.join(old_labels_dir, file)
                shutil.copy(src_path, dst_path)
                print(f"Copied label {src_path} -> {dst_path}")
    else:
        print(f"Labels directory not found for split '{split}' in new dataset.")

print("New dataset merged successfully into old dataset.\n")

# ------------------ שלב 3: יצירת מבנה סופי (חלוקה ל-train, valid, test) ------------------
print("Step 3: Splitting merged dataset into train, valid, and test sets...")

# נניח שהדאטה הממוזגת נמצאת ב-old_dataset (המשולב)
final_root = "/kaggle/working/final_dataset"
for split in splits:
    os.makedirs(os.path.join(final_root, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(final_root, split, "labels"), exist_ok=True)

# איסוף כל קבצי התמונות ממיזוג הדאטה
merged_images_dir = os.path.join(old_dataset_dir, "train", "images")  # נניח שמיזוג התמונות בוצע ב-train
all_images = glob.glob(os.path.join(merged_images_dir, "*"))
random.shuffle(all_images)
total = len(all_images)
train_count = math.floor(total * 0.8)
valid_count = math.floor(total * 0.1)

train_images = all_images[:train_count]
valid_images = all_images[train_count:train_count+valid_count]
test_images = all_images[train_count+valid_count:]

def copy_split(image_list, split):
    for img_path in image_list:
        filename = os.path.basename(img_path)
        dest_img = os.path.join(final_root, split, "images", filename)
        shutil.copy(img_path, dest_img)
        print(f"[{split}] Copied image: {filename}")
        # העתקת תיוגים – מניחים שקובץ התיוג שם זהה עם סיומת .txt
        label_filename = os.path.splitext(filename)[0] + ".txt"
        src_label = os.path.join(os.path.dirname(img_path).replace("images", "labels"), label_filename)
        if os.path.exists(src_label):
            dest_label = os.path.join(final_root, split, "labels", label_filename)
            shutil.copy(src_label, dest_label)
            print(f"[{split}] Copied label: {label_filename}")
        else:
            print(f"[{split}] Label not found for image: {filename}")

copy_split(train_images, "train")
copy_split(valid_images, "valid")
copy_split(test_images, "test")
print("Dataset successfully split into train, valid, and test sets.\n")

# ------------------ שלב 4: יצירת קובץ YAML ------------------
print("Step 4: Writing dataset YAML file...")
data_yaml = {
    "train": "../train/images",
    "val": "../valid/images",
    "test": "../test/images",
    "nc": 11,
    "names": [
        "Alligator Crack",
        "Block Crack",
        "Construction Joint Crack",
        "Crosswalk Blur",
        "Lane Blur",
        "Longitudinal Crack",
        "Manhole",
        "Patch Repair",
        "Pothole",
        "Transverse Crack",
        "Wheel Mark Crack"
    ],
    "roboflow": {
        "workspace": "devflo",
        "project": "rdd2022-ook7x",
        "version": 3,
        "license": "CC BY 4.0",
        "url": "https://universe.roboflow.com/devflo/rdd2022-ook7x/dataset/3"
    }
}

yaml_path = os.path.join(final_root, "dataset.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)
print("Dataset YAML file written to:", yaml_path)
print("All steps completed successfully!")


# ייצוא מודל יולו לפורמט אחר 


In [ ]:
import onnx2tf
!pip install tflite-support

model = YOLO("yolo12n.pt")
model = YOLO("/kaggle/working/road_safety_model/weights/best.pt")
model.export(format="tflite",data = "/kaggle/working/final_dataset/dataset.yaml")

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

MODEL_PATH = "/kaggle/working/road_safety_model/weights/best_saved_model/best_float32.tflite"
LABEL_FILE = "/kaggle/working/road_safety_model/weights/best_saved_model/metadata.yaml"
EXPORT_DIR = "/kaggle/working/road_safety_model/weights"

# יוצר את metadata עם נרמול
writer = object_detector.MetadataWriter.create_for_inference(
    model_buffer=open(MODEL_PATH, "rb").read(),
    input_norm_mean=[127.5],
    input_norm_std=[127.5],
    label_file_paths=[LABEL_FILE]
)

model_with_metadata = writer.populate()
writer_utils.save_file(model_with_metadata, f"{EXPORT_DIR}/new_model_with_metadata.tflite")

print("✅ Metadata added! Saved as new_model_with_metadata.tflite")


In [ ]:
import os

# הנתיב הראשי שהגדרת באימון
project_dir = "/kaggle/working"

# חיפוש בתיקיות שבתוך הפרויקט
folders = [os.path.join(project_dir, f) for f in os.listdir(project_dir) if os.path.isdir(os.path.join(project_dir, f))]

# פילטר לפי תיקיות YOLO עם קבצי last.pt
yolo_dirs = []
for folder in folders:
    last_pt_path = os.path.join(folder, "weights", "last.pt")
    if os.path.exists(last_pt_path):
        yolo_dirs.append((folder, os.path.getmtime(last_pt_path)))  # שמירה עם זמן עדכון

# מיון לפי זמן שמירה (האחרון למעלה)
yolo_dirs.sort(key=lambda x: x[1], reverse=True)

# הדפסת הנתיב האחרון אם קיים
if yolo_dirs:
    last_folder = yolo_dirs[0][0]
    print("📁 YOLO last training folder found at:")
    print(last_folder)
    print("🔥 last.pt path:")
    print(os.path.join(last_folder, "weights", "last.pt"))
else:
    print("לא נמצא קובץ last.pt בסביבת העבודה.")


In [ ]:
# התקנת ספריות
%pip install ultralytics

# יבוא מודול + בדיקת מערכת
from ultralytics import YOLO, checks





# יבוא מודול + בדיקת מערכת
from ultralytics import YOLO, checks


model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")
# התחלת האימון
results = model.train(
    data="/kaggle/working/final_dataset/dataset.yaml",  # הנתיב המלא לקובץ הנתונים שלך
    epochs=50,                  # יותר אפוקים ללמידה טובה בתנאים קשים
    imgsz=540,                   # רזולוציה גבוהה – מאפשרת זיהוי פרטים קטנים
    batch=0.60,
    patience=15,
    optimizer='SGD',
    lr0=0.002,
    lrf=0.01,
    seed=42,
    amp=True,
    cache="disk",
    workers=8,
    cos_lr=True,
    warmup_epochs=3,
    verbose=True,
    save=True,
    profile	=True,
    resume  = True,
    device = 0
)



Note: you may need to restart the kernel to use updated packages.
Ultralytics 8.3.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/new_model/tensorflow2/default/1/best (9).pt, data=/kaggle/working/final_dataset/dataset.yaml, epochs=50, time=None, patience=15, batch=0.6, imgsz=540, save=True, save_period=-1, cache=disk, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=True, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

train: Scanning /kaggle/working/final_dataset/train/labels.cache... 66540 images, 19812 backgrounds, 0 corrupt: 100%|██████████| 66540/66540 [00:00<?, ?it/s]


train: 114.2GB disk space required, with 50% safety margin but only 8.9/19.5GB free, not caching images to disk ⚠️
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=544 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.74G total, 13.07G reserved, 0.09G allocated, 1.58G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2570193       4.689         1.812         50.36          42.5        (1, 3, 544, 544)                    list
     2570193       9.379         1.850         42.94         45.69        (2, 3, 544, 544)                    list
     2570193       18.76         1.883          45.2         47.28        (4, 3, 544, 544)                    list
     2570193       37.52         2.

train: Scanning /kaggle/working/final_dataset/train/labels.cache... 66540 images, 19812 backgrounds, 0 corrupt: 100%|██████████| 66540/66540 [00:00<?, ?it/s]


train: 114.2GB disk space required, with 50% safety margin but only 8.9/19.5GB free, not caching images to disk ⚠️
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

       4/50      2.27G       1.52      1.508      1.362         32        544: 100%|██████████| 4159/4159 [13:39<00:00,  5.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 260/260 [00:48<00:00,  5.32it/s]


                   all       8317      14642      0.696      0.693      0.741      0.484

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 260/260 [00:49<00:00,  5.29it/s]


                   all       8317      14642      0.734      0.694      0.753      0.487

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/50      2.27G       1.51      1.494      1.353         26        544:  15%|█▍        | 620/4159 [02:02<11:35,  5.09it/s]

In [ ]:
!ls /content/drive/MyDrive/saved_models/road_safety_model


# בדיקת מודל על תמונה תיקייה או סרטון

##משיכת מודל שמור מהדרייב אם יש

In [ ]:
import os
import glob
from google.colab import drive

# חיבור ל-Google Drive
drive.mount('/content/drive')

# נתיב לתיקיית המודלים
models_dir = "/content/drive/MyDrive/saved_models"

# רשימת סיומות חוקיות למודלים
valid_exts = [".pt", ".onnx", ".pth"]

# חיפוש כל הקבצים החוקיים
model_files = []
for ext in valid_exts:
    model_files.extend(glob.glob(os.path.join(models_dir, f"*{ext}")))

# בדיקה אם נמצאו מודלים
if not model_files:
    print("😕 No model files found in the folder.")
else:
    print("📦 Available models:\n")
    for i, path in enumerate(model_files):
        print(f"{i + 1}. {os.path.basename(path)}")

    # בחירת מודל
    while True:
        try:
            choice = int(input("\n🔍 Enter the number of the model you want to load: ")) - 1
            if 0 <= choice < len(model_files):
                selected_model = model_files[choice]
                print(f"\n✅ You selected: {selected_model}")
                break
            else:
                print("❌ Invalid choice. Try again.")
        except ValueError:
            print("⚠️ Please enter a valid number.")


In [ ]:
%pip install ultralytics
from ultralytics import YOLO
model = YOLO("/road_damage_detection_last_version.pt")
results = model.predict(
    source='/content/drive/MyDrive/test_me/videos_only/IMG_6205.mov',
    save=True,
    save_txt=False,
    conf=0.25
)


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BQvvxKrJPoAaNkv4ZXlr")
project = rf.workspace("laura-hjwf7").project("pothole-at-night")
version = project.version(1)
dataset = version.download("yolov12")
